In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


C:\Users\Anadu\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load datasets
resumes = pd.read_csv('data/Resume Dataset/Resume/Resume_With_Skills.csv')
jobs = pd.read_csv('data/Linkedin Job Postings (2023-2024)/cleaned_JD_with_skills.csv')

# print column names
print(resumes.columns)
print(jobs.columns)




Index(['ID', 'Resume_str', 'Resume_html', 'Category', 'Clean_Resume', 'skills',
       'soft_skills'],
      dtype='object')
Index(['title', 'description', 'skills_desc', 'combined_skills_desc',
       'Clean_JD', 'skills'],
      dtype='object')


In [5]:
resumes["text"] = (
    resumes["Clean_Resume"] + " " +
    resumes["skills"].fillna("") + " " +
    resumes["soft_skills"].fillna("")
)

jobs["text"] = (
    jobs["Clean_JD"] + " " +
    jobs["skills"].fillna("") + " " +
    jobs["skills_desc"].fillna("")
)

In [6]:
# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

c:\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anadu\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Anadu\AppData\Roaming\Python\Python310\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoint

In [13]:
# Generate embeddings
def get_embeddings(text_list):
    inputs = tokenizer(
        text_list,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()



In [14]:
resumes["text"] = resumes["text"].astype(str).fillna("")
jobs["text"] = jobs["text"].astype(str).fillna("")

# Compute embeddings
resume_embeddings = get_embeddings(resumes["text"].tolist())
job_embeddings = get_embeddings(jobs["text"].tolist())

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 976748544 bytes.

# Rank Jobs

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def rank_jobs(resume_text, job_embeddings, jobs, top_n=10):
    # Embed the resume
    resume_embedding = get_embeddings([resume_text])
    
    # Compute cosine similarity
    similarities = cosine_similarity(resume_embedding, job_embeddings)
    
    # Get top N jobs
    top_indices = similarities.argsort()[0][-top_n:][::-1]
    top_scores = similarities[0, top_indices]
    
    return jobs.iloc[top_indices], top_scores

# Example resume
resume_text = resumes["text"].iloc[0]  # Use the first resume for testing
top_jobs, scores = rank_jobs(resume_text, job_embeddings, jobs)

# Display results
print("Top Job Matches:")
for job, score in zip(top_jobs["title"], scores):
    print(f"{job}: {score:.2f}")
